## Vergleich des Verhaltens von Politikern in Sozialen Medien

In [ ]:
library(lubridate)
library(ggplot2)
library(dplyr)
library(readr)
library(tidytext)
library(stringr)
source("SentiWS_v1.8c/SentiWS.R")

In [ ]:
tweets_1.csv <- read_csv("twitter_output/twitter_1.csv")
tweets_2.csv <- read_csv("twitter_output/twitter_2.csv")

In [ ]:
tweets <- bind_rows(tweets_1.csv %>% mutate(collection = "Bernd Riexinger"),
                    tweets_2.csv %>% mutate(collection = "Alice Weidel")) 

In [ ]:
head(tweets[, c("date", "username", "text")], n = 5)

In [ ]:
ggplot(tweets, aes(x = date, fill = collection)) +
  geom_histogram(position = "identity", bins = 20, show.legend = FALSE) +
  facet_wrap(~collection, ncol = 1)

## Sentiment Analyse

In [ ]:
SentiWS <- read.SentiWS()

head(SentiWS, n = 5)